<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Neural Network Framework (Keras)

## *Data Science Unit 4 Sprint 2 Assignment 3*

## Use the Keras Library to build a Multi-Layer Perceptron Model on the Boston Housing dataset

- The Boston Housing dataset comes with the Keras library so use Keras to import it into your notebook. 
- Normalize the data (all features should have roughly the same scale)
- Import the type of model and layers that you will need from Keras.
- Instantiate a model object and use `model.add()` to add layers to your model
- Since this is a regression model you will have a single output node in the final layer.
- Use activation functions that are appropriate for this task
- Compile your model
- Fit your model and report its accuracy in terms of Mean Squared Error
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Run this same data through a linear regression model. Which achieves higher accuracy?
- Do a little bit of feature engineering and see how that affects your neural network model. (you will need to change your model to accept more inputs)
- After feature engineering, which model sees a greater accuracy boost due to the new features?

In [24]:
##### Your Code Here #####
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import boston_housing
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [4]:
print(f"The shape of the x train is:  {x_train.shape}")
print(f"The shape of the y_train is: {y_train.shape}")
print(f"The shape of the x_test is : {x_test.shape}")
print(f"The shape of the y_test is: {y_test.shape}")
# Will not need to flatten anything becuase it is only a two dimensional array

The shape of the x train is:  (404, 13)
The shape of the y_train is: (404,)
The shape of the x_test is : (102, 13)
The shape of the y_test is: (102,)


In [5]:
# looking at the shape of one row of the data
# and what is in it
x_train[:1]

array([[  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
         91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
         18.72   ]])

In [6]:
type(x_train)

numpy.ndarray

In [7]:
# function that will do the normalization
def normalization(num_array):
     arr =  num_array/np.amax(num_array, axis=0)
     return arr

In [8]:
# Doing the normalization
x_train_norm = normalization(x_train)
x_test_norm = normalization(x_test)

In [10]:
# Looking at the difference between the non normalized and the 
# normalized version
x_train[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [11]:
x_test_norm[0]

array([0.72205254, 0.        , 0.65248738, 0.        , 0.77956372,
       0.73280182, 1.        , 0.15129675, 1.        , 0.93670886,
       0.95283019, 0.06865709, 0.90809628])

In [12]:
# Instanciating the model
model = Sequential()

In [51]:
# getting the callback function that I want to use
the_callback = EarlyStopping(patience=1, monitor="loss")

In [82]:
# Builing the model
# adding layers to the model
model.add(Dense(64, input_shape=(13,), activation="relu"))
model.add(Dense(64, activation="relu"))
#model.add(Dense(128, activation="relu"))
#model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="linear"))

# Doing the compiliation of the model
model.compile(optimizer="adam", loss="mse", metrics=['mse', "mae", ], callbacks=[the_callback] )

In [72]:
# looking at the summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 26)                364       
_________________________________________________________________
dense_1 (Dense)              (None, 13)                351       
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 98        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 8         
_________________________________________________________________
dense_4 (Dense)              (None, 26)                52        
_________________________________________________________________
dense_5 (Dense)              (None, 13)                351       
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 9

In [85]:
# Doing the fitting of the model
model.fit(x=x_train_norm, y=y_train, epochs=10,  validation_data=(x_test_norm, y_test)
) 

Train on 404 samples, validate on 102 samples
Epoch 1/10
404/404 [==============================] - 0s 319us/sample - loss: 6.6618 - mse: 6.6618 - mae: 1.9164 - val_loss: 41.6104 - val_mse: 41.6104 - val_mae: 5.0742
Epoch 2/10
404/404 [==============================] - 0s 351us/sample - loss: 8.0327 - mse: 8.0327 - mae: 2.0819 - val_loss: 29.6711 - val_mse: 29.6711 - val_mae: 3.8374
Epoch 3/10
404/404 [==============================] - 0s 347us/sample - loss: 5.7822 - mse: 5.7822 - mae: 1.7712 - val_loss: 27.7126 - val_mse: 27.7126 - val_mae: 3.6473
Epoch 4/10
404/404 [==============================] - 0s 332us/sample - loss: 6.5720 - mse: 6.5720 - mae: 1.9249 - val_loss: 40.2494 - val_mse: 40.2494 - val_mae: 4.9329
Epoch 5/10
404/404 [==============================] - 0s 356us/sample - loss: 8.5346 - mse: 8.5346 - mae: 2.2752 - val_loss: 30.4549 - val_mse: 30.4549 - val_mae: 3.9000
Epoch 6/10
404/404 [==============================] - 0s 389us/sample - loss: 6.1204 - mse: 6.1204 - mae

In [ ]:
# Will be training the model


## Use the Keras Library to build an image recognition network using the Fashion-MNIST dataset (also comes with keras)

- Load and preprocess the image data similar to how we preprocessed the MNIST data in class.
- Make sure to one-hot encode your category labels
- The number of nodes in your output layer should equal the number of classes you want to predict for Fashion-MNIST.
- Try different hyperparameters. What is the highest accuracy that you are able to achieve.
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Remember that neural networks fall prey to randomness so you may need to run your model multiple times (or use Cross Validation) in order to tell if a change to a hyperparameter is truly producing better results.

In [ ]:
##### Your Code Here #####

## Stretch Goals:

- Use Hyperparameter Tuning to make the accuracy of your models as high as possible. (error as low as possible)
- Use Cross Validation techniques to get more consistent results with your model.
- Use GridSearchCV to try different combinations of hyperparameters. 
- Start looking into other types of Keras layers for CNNs and RNNs maybe try and build a CNN model for fashion-MNIST to see how the results compare.